In [80]:
from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
# llm = OpenAI(temperature=0, verbose=True)

# sql_agent的主要不同点是加入了suffix和prefix, 见 langchain\agents\agent_toolkits\sql\prompt.py
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0, model = "gpt-4"),
    toolkit=SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0),handle_parsing_errors=True),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

In [ ]:
agent_executor.run("How many employees are there?")

TODO
1. 加入memory : 创建 agent_chain...
2. 替换 tools

### 加入memory

In [86]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, ConversationalChatAgent, ConversationalAgent
from langchain.utilities import SerpAPIWrapper
from langchain.memory import ConversationBufferMemory




prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools，If there is no tool useful, answer it by yourself instead of Action:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""
search = SerpAPIWrapper()
tools = [
    
]

prompt = ConversationalAgent.create_prompt(
    tools,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory=ConversationBufferMemory(memory_key="chat_history", input_key='input', output_key="output",return_messages = True)


In [87]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=prompt)
agent = ConversationalAgent(llm_chain=llm_chain, tools=tools, verbose=True, output_parser = JSONAgentOutputParser())
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True,return_intermediate_steps=True, stop = ""
)

In [88]:
from langchain.schema import HumanMessage
from qunar import http_interceptor
from langchain.prompts.chat import HumanMessagePromptTemplate
http_interceptor.init()
p = HumanMessage(content="生成一段sql, 有关查询班级的总人数")
import asyncio

response = agent_chain.invoke(p)
# agent_chain.stream()
# response["intermediate_steps"]




> Entering new AgentExecutor chain...
Could not parse LLM output: Thought: Do I need to use a tool? Yes
Action: code_generation
Action Input: content='生成一段sql, 有关查询班级的总人数'
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Do I need to use a tool? Yes
Observation: Invalid or incomplete response
Thought:Could not p

ValidationError: 1 validation error for HumanMessage
content
  str type expected (type=type_error.str)

这里出现了一个问题，tool报错导致chain执行失败，在写tool的时候应该返回错误码

In [61]:
memory.load_memory_variables([])

{'chat_history': 'Human: 生成一段sql\nAI: I am sorry, but I am not able to generate a SQL statement.'}